In [15]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

from keras import layers
from keras import models
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array

In [ ]:
%tensorflow_version 1.x
import tensorflow as tf

# Download the code
!git clone https://github.com/NVlabs/stylegan2.git
%cd stylegan2
!nvcc test_nvcc.cu -o test_nvcc -run

In [2]:
#Create the architecture of the model
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])


In [5]:
#Get the sample data
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_dir = '/Users/mherm/Google Drive/glasses_work_folder/training'
validation_dir = '/Users/mherm/Google Drive/glasses_work_folder/validation'

#Turn it into tensors
train_generator = train_datagen.flow_from_directory(
        # This is the target directory
        train_dir,
        # All images will be resized to 150x150
        target_size=(150, 150),
        batch_size=20,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')


Found 4000 images belonging to 2 classes.
Found 1989 images belonging to 2 classes.


In [6]:
#Train the network
history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=30,
      validation_data=validation_generator,
      validation_steps=50)


Epoch 1/30
100/100 [==============================] - 99s 991ms/step - loss: 0.4758 - acc: 0.7705 - val_loss: 0.3589 - val_acc: 0.8900
Epoch 2/30
100/100 [==============================] - 114s 1s/step - loss: 0.2164 - acc: 0.9245 - val_loss: 0.3037 - val_acc: 0.9272
Epoch 3/30
100/100 [==============================] - 117s 1s/step - loss: 0.1865 - acc: 0.9485 - val_loss: 0.2264 - val_acc: 0.9560
Epoch 4/30
100/100 [==============================] - 115s 1s/step - loss: 0.1611 - acc: 0.9520 - val_loss: 0.0590 - val_acc: 0.9525
Epoch 5/30
100/100 [==============================] - 134s 1s/step - loss: 0.1466 - acc: 0.9615 - val_loss: 0.0281 - val_acc: 0.9540
Epoch 6/30
100/100 [==============================] - 149s 1s/step - loss: 0.1490 - acc: 0.9560 - val_loss: 1.0065 - val_acc: 0.9596
Epoch 7/30
100/100 [==============================] - 156s 2s/step - loss: 0.1486 - acc: 0.9575 - val_loss: 0.0945 - val_acc: 0.9600
Epoch 8/30
100/100 [==============================] - 164s 2s/step 

In [7]:
#Save the model afterwards
model.save("glasses-5-17-2019.h5")

In [32]:
#Evaulate the model on some images
img = load_img('/Users/mherm/Google Drive/glasses_work_folder/examples/example8.jfif', target_size=(150,150))
img.show()
input_arr = np.array([img_to_array(img)])
model.predict(input_arr)

array([[0.]], dtype=float32)